In [2]:
import os
import pandas as pd
import numpy as np

import pyreadr

#import dask_mpi as dm
from dask.distributed import Client, progress
import dask
from dask.distributed import performance_report

from datetime import datetime
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
import joblib

In [2]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
DP02_location = "/home/usr6/q70176a/DP02/"
DP02_result_location = "/home/usr6/q70176a/DP02/08_PyResults/"

In [2]:
DP02_location = "/Users/lichao/Library/CloudStorage/OneDrive-KyushuUniversity/02_Article/03_RStudio/"
DP02_result_location = "/Users/lichao/Library/CloudStorage/OneDrive-KyushuUniversity/02_Article/03_RStudio/08_PyResults/"

In [3]:
DP02_location = "D:/OneDrive - Kyushu University/02_Article/03_RStudio/"
DP02_result_location = "D:/OneDrive - Kyushu University/02_Article/03_RStudio/08_PyResults/"

In [4]:
client = Client(n_workers=3, nthreads=1)
#client = Client(n_workers=4, threads_per_worker=1)

D:\Anaconda3\envs\Usegdal\lib\site-packages\distributed\node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57864 instead
  warnings.warn(


In [5]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:57864/status,
Dashboard: http://127.0.0.1:57864/status,Workers: 2
Total threads: 12,Total memory: 63.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57865,Workers: 2
Dashboard: http://127.0.0.1:57864/status,Total threads: 12
Started: Just now,Total memory: 63.92 GiB
Comm: tcp://127.0.0.1:57886,Total threads: 6
Dashboard: http://127.0.0.1:57887/status,Memory: 31.96 GiB
Nanny: tcp://127.0.0.1:57868,


In [6]:
dataset = pyreadr.read_r(DP02_location + "02_Data/SP_Data_49Variable_Weights_changeRangeOfLandCover_RdsVer.Rds")
dataset = dataset[None]
y = np.array(dataset.iloc[:, 0:1].values.flatten(), dtype='float64')
X = np.array(dataset.iloc[:, 1:50], dtype='float64')

In [7]:
%%time
model = RandomForestRegressor(n_estimators=1000, oob_score=True, 
                               random_state=1, max_features = 9, n_jobs=-1)
with joblib.parallel_backend("dask"): model.fit(X, y)

Wall time: 1min 13s


In [8]:
# SHAP
import dalex as dx
model_rf_exp = dx.Explainer(model, X, y, label = "RF Pipeline")

Preparation of a new explainer is initiated

  -> data              : numpy.ndarray converted to pandas.DataFrame. Columns are set as string numbers.
  -> data              : 89273 rows 49 cols
  -> target variable   : 89273 values
  -> model_class       : sklearn.ensemble._forest.RandomForestRegressor (default)
  -> label             : RF Pipeline
  -> predict function  : <function yhat_default at 0x000002B2004CBBE0> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.


D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


  -> predicted values  : min = 1.18, mean = 24.3, max = 36.0
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)


D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


  -> residuals         : min = -11.2, mean = 0.0166, max = 7.13
  -> model_info        : package sklearn

A new explainer has been created!


In [9]:
with performance_report(filename="dask-report-scatter.html"):
    X_scattered = client.scatter(X)
    model_rf_exp_scattered = client.scatter(model_rf_exp)

In [10]:
def singleSHAPprocess(obs_num, X, model_rf_exp):
    test_obs = X[obs_num:obs_num+1,:]
    shap_test = model_rf_exp.predict_parts(test_obs, type = 'shap', 
                                           B = 5, N = 100)
    result = shap_test.result[shap_test.result.B == 0]
    result = result[['contribution', 'variable_name']]
    result = result.transpose()
    result = result.rename(columns=result.iloc[1])
    result = result.drop(['variable_name'], axis=0)
    result = result.reset_index(drop=True)
    return result

In [11]:
%%time
delayed_results = [dask.delayed(singleSHAPprocess)(obs_num, X_scattered, model_rf_exp_scattered)  for obs_num in list(range(40))]
#results = client.compute(*delayed_results)

Wall time: 2.03 ms


In [17]:
delayed_results[1]

Delayed('singleSHAPprocess-106124c2-6411-434e-806a-15c8c0e70708')

In [13]:
with performance_report(filename="dask-report-shap.html"):
    %time results = dask.compute(*delayed_results)

Wall time: 42min 49s


In [14]:
client.close()

In [15]:
%%time
singleSHAPprocess(1, X, model_rf_exp)

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

Wall time: 34 s


D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


,18,10,30,14,13,2,31,12,23,36,...,5,7,26,21,27,6,8,4,34,25
0,-2.512683,-2.418323,-1.164454,-1.069467,-0.627867,-0.513087,-0.493534,0.464189,-0.375833,-0.359282,...,0.005376,0.004328,0.003786,-0.002992,0.001724,0.0016,0.000934,0.000282,0.00013,0.00003


In [7]:
import shap

In [8]:
explainer = shap.TreeExplainer(model)

In [27]:
choosen_instance = X[0]

In [28]:
choosen_instance

array([-5.18236813e-01,  3.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  2.00000000e+00,  3.00000000e+00,
        2.00000000e+00,  3.00000000e+00,  2.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        3.00000000e+00,  0.00000000e+00,  7.40000000e+01,  3.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  0.00000000e+00,  4.00000000e+00,
        3.00000000e+00,  3.00000000e+00,  3.00000000e+00,  1.45524730e+01,
        2.72555890e+01,  3.70769380e+00,  5.20693060e-01,  3.59098650e-02,
        5.47625480e-01,  5.32004660e+01,  1.79549340e-01,  1.35410890e+02,
        3.48724300e+01])

In [29]:
%%time
shap_values = explainer.shap_values(choosen_instance)

Wall time: 1min 15s


In [30]:
shap_values

array([ 7.43196534e-02,  2.51093661e-01,  6.48591676e-03,  3.41867736e-02,
        2.57996614e-04,  6.20038464e-03, -2.96505123e-03,  4.74986043e-03,
        6.77891018e-04, -2.91178281e-03, -1.17089094e+00, -2.30796680e-01,
        1.07815237e+00,  1.16038460e-01, -9.17292786e-01, -1.07218565e-01,
        2.67317893e-02, -6.43699592e-02, -1.26986813e+00, -9.73541458e-04,
       -2.18591289e-02, -1.26346285e-02,  3.72171250e-02, -3.04020746e-01,
        3.18489894e-02,  2.12211261e-03,  5.30907411e-03,  3.17641742e-03,
        2.16095366e-01,  2.76668657e-02,  8.07168431e-01, -9.47360842e-01,
        2.31170752e-02, -1.17450043e-01,  3.63615657e-03,  2.65001742e-02,
       -1.72248141e-01, -6.37874713e-03,  5.86502895e-02, -6.30689025e-03,
       -3.18063532e-02, -4.35267903e-02,  4.65947125e-02,  4.90376181e-03,
        1.85027094e-02, -6.65257683e-03,  2.65403958e-02,  1.39525483e-01,
       -6.30227711e-02])

In [31]:
%%time
#shap_values_ori=singleSHAPprocess(0, X, model_rf_exp) 

Wall time: 0 ns


In [34]:
shap_values_ori

,18,12,30,10,14,31,23,11,1,36,...,2,27,5,9,8,39,21,4,6,34
0,-1.18125,0.710321,0.676705,-0.673839,-0.648899,-0.623327,-0.260915,-0.199091,0.167619,-0.108738,...,0.005526,0.003872,0.002598,0.002307,0.00195,-0.001882,0.001535,0.001006,-0.000522,0.0


In [9]:
run = explainer.shap_values

In [40]:
%%time
run(choosen_instance)

Wall time: 49 s


array([ 7.43196534e-02,  2.51093661e-01,  6.48591676e-03,  3.41867736e-02,
        2.57996614e-04,  6.20038464e-03, -2.96505123e-03,  4.74986043e-03,
        6.77891018e-04, -2.91178281e-03, -1.17089094e+00, -2.30796680e-01,
        1.07815237e+00,  1.16038460e-01, -9.17292786e-01, -1.07218565e-01,
        2.67317893e-02, -6.43699592e-02, -1.26986813e+00, -9.73541458e-04,
       -2.18591289e-02, -1.26346285e-02,  3.72171250e-02, -3.04020746e-01,
        3.18489894e-02,  2.12211261e-03,  5.30907411e-03,  3.17641742e-03,
        2.16095366e-01,  2.76668657e-02,  8.07168431e-01, -9.47360842e-01,
        2.31170752e-02, -1.17450043e-01,  3.63615657e-03,  2.65001742e-02,
       -1.72248141e-01, -6.37874713e-03,  5.86502895e-02, -6.30689025e-03,
       -3.18063532e-02, -4.35267903e-02,  4.65947125e-02,  4.90376181e-03,
        1.85027094e-02, -6.65257683e-03,  2.65403958e-02,  1.39525483e-01,
       -6.30227711e-02])

In [18]:
client = Client(n_workers=2, nthreads=1)

In [19]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 12,Total memory: 63.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51390,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 63.92 GiB
Comm: tcp://127.0.0.1:51424,Total threads: 6
Dashboard: http://127.0.0.1:51425/status,Memory: 31.96 GiB
Nanny: tcp://127.0.0.1:51393,


In [10]:
run_scattered = client.scatter(run)

distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp

distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp

In [11]:
X_scattered = client.scatter(X)

In [25]:
def singleSHAP(obs_num, X, run):
    test_obs = X[obs_num:obs_num+1,:]
    result = run(test_obs)
    return result

In [55]:
%%time
delayed_results = [dask.delayed(singleSHAP)(obs_num, X_scattered, run_scattered)  for obs_num in list(range(1000))]
#results = client.compute(*delayed_results)

Wall time: 47 ms


In [56]:
%%time
results = dask.compute(*delayed_results)

2022-11-14 21:09:38,994 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:58834', name: 0, status: running, memory: 176, processing: 326>
2022-11-14 21:09:58,159 - distributed.nanny - WARNING - Worker process still alive after 3.999998664855957 seconds, killing
2022-11-14 23:04:39,021 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:58831', name: 1, status: running, memory: 317, processing: 685>


CancelledError: singleSHAP-998ea65e-2ad4-4890-823b-d362f780164e

2022-11-14 23:04:58,095 - distributed.nanny - WARNING - Worker process still alive after 3.999998664855957 seconds, killing


In [17]:
client.close()

## To HPC Test Memory

In [1]:
import os
import pandas as pd
import numpy as np

import pyreadr

#import dask_mpi as dm
from dask.distributed import Client, progress, get_task_stream
import dask

from datetime import datetime
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
import joblib

DP02_location = "/home/usr6/q70176a/DP02/"
DP02_result_location = "/home/usr6/q70176a/DP02/08_PyResults/"

client = Client(n_workers=3, threads_per_worker=1)
print(client)

dataset = pyreadr.read_r(DP02_location + "02_Data/SP_Data_49Variable_Weights_changeRangeOfLandCover_RdsVer.Rds")
dataset = dataset[None]
y = np.array(dataset.iloc[:, 0:1].values.flatten(), dtype='float64')
X = np.array(dataset.iloc[:, 1:50], dtype='float64')


model = RandomForestRegressor(n_estimators=1000, oob_score=True, 
                               random_state=1, max_features = 9, n_jobs=-1)
from dask.distributed import performance_report
with performance_report(filename="dask-report.html"):
    with joblib.parallel_backend("dask"): model.fit(X, y)

<Client: 'tcp://127.0.0.1:56955' processes=3 threads=3, memory=63.92 GiB>


PyreadrError: File b'/home/usr6/q70176a/DP02/02_Data/SP_Data_49Variable_Weights_changeRangeOfLandCover_RdsVer.Rds' does not exist!

In [9]:
import shap
explainer = shap.TreeExplainer(model)
def singleSHAP(obs_num, X, explainer):
    test_obs = X[obs_num:obs_num+1,:]
    result = explainer.shap_values(test_obs)
    return result


In [13]:
with performance_report(filename="dask-report-shap.html"):
    delayed_results = [dask.delayed(singleSHAP)(obs_num, X, explainer)  for obs_num in list(range(20))]
    results = dask.compute(*delayed_results)

Wall time: 8min 35s


In [ ]:
with performance_report(filename="dask-report-scatter.html"):
    X_scattered = client.scatter(X)
    explainer_scattered = client.scatter(explainer)

In [ ]:
test = client.submit(singleSHAP, 0, X_scattered, run_scattered)

In [26]:
seriesTest = []
for obs in list(range(20)):
    test = client.submit(singleSHAP, obs, X_scattered, run_scattered)
    seriesTest.append(test)

In [27]:
client.gather(seriesTest)

Wall time: 30min 53s


In [30]:
client.close()

In [ ]:
with performance_report(filename="dask-report-shap.html"):
    delayed_results = [dask.delayed(singleSHAP)(obs_num, X_scattered, run_scattered)  for obs_num in list(range(20))]
    results = dask.compute(*delayed_results)